In [1]:
# Install required libraries
!pip install geopandas shapely openpyxl folium

In [2]:
# Import all required libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import folium
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


In [3]:
# Upload the shapefile components
print("Please upload ALL 6 shapefile files for statistical areas:")
print("- statistical_areas_2022.shp")
print("- statistical_areas_2022.shx")
print("- statistical_areas_2022.dbf")
print("- statistical_areas_2022.prj")
print("- statistical_areas_2022.cpg")
print("- statistical_areas_2022.xml")
print()

uploaded = files.upload()

Please upload ALL 6 shapefile files for statistical areas:
- statistical_areas_2022.shp
- statistical_areas_2022.shx
- statistical_areas_2022.dbf
- statistical_areas_2022.prj
- statistical_areas_2022.cpg
- statistical_areas_2022.xml



Saving statistical_areas_2022.cpg to statistical_areas_2022.cpg
Saving statistical_areas_2022.dbf to statistical_areas_2022.dbf
Saving statistical_areas_2022.prj to statistical_areas_2022.prj
Saving statistical_areas_2022.shp to statistical_areas_2022.shp
Saving statistical_areas_2022.shx to statistical_areas_2022.shx
Saving statistical_areas_2022.xml to statistical_areas_2022.xml


In [4]:
# Load statistical areas shapefile
gdf_areas = gpd.read_file('statistical_areas_2022.shp')

print(f"✓ Loaded {len(gdf_areas)} statistical areas")
print(f"✓ CRS: {gdf_areas.crs}")
print(f"✓ Statistical area codes: {sorted(gdf_areas['STAT_2022'].unique())}")

✓ Loaded 25 statistical areas
✓ CRS: EPSG:2039
✓ Statistical area codes: [np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(21), np.int32(22), np.int32(23), np.int32(24), np.int32(25), np.int32(26), np.int32(31), np.int32(32), np.int32(33), np.int32(41), np.int32(42), np.int32(43), np.int32(44), np.int32(45), np.int32(46), np.int32(47)]


In [5]:
# Upload Restaurants CSV file
print("Please upload Eilat_restaurants_data_2025.csv")
uploaded = files.upload()

# Load Restaurants data
restaurants = pd.read_csv('Eilat_restaurants_data_2025.csv')

print(f"\n✓ Loaded {len(restaurants)} restaurant records")
print(f"\nColumns: {restaurants.columns.tolist()}")
print("\nFirst few rows:")
print(restaurants.head(10))

# Check data types
print("\n" + "="*80)
print("Data types:")
print(restaurants.dtypes)

Please upload Eilat_restaurants_data_2025.csv


Saving Eilat_restaurants_data_2025.csv to Eilat_restaurants_data_2025.csv

✓ Loaded 280 restaurant records

Columns: ['cid', 'title', 'description', 'categoryName', 'totalScore', 'temporarilyClosed', 'permanentlyClosed', 'url', 'website', 'street', 'location/lat', 'location/lng', 'Activity times']

First few rows:
                    cid                                              title  \
0  13748798895203813526                                  Burger King Ramon   
1  11503681463165581249                             בורגרים שדה תעופה רמון   
2   7627013372514451195                                             Omer's   
3  12420276880766813630                                      Pastory Eilat   
4   3210688865794418088                                  Branja Restaurant   
5   1361957468337808507                                        The Brewery   
6   6440275474264107117  Tikka Masala - טיקה מסאלה אילת | מסעדה הודית ב...   
7  10738057538163195435                                     

In [6]:
print("Examining restaurants data...\n")
print("="*80)

# Check for missing coordinates
missing_coords = restaurants[
    restaurants['location/lat'].isna() |
    restaurants['location/lng'].isna()
]

print(f"Records with missing coordinates: {len(missing_coords)}")
print(f"Records with valid coordinates: {len(restaurants) - len(missing_coords)}")

# Check coordinate ranges
print("\n" + "="*80)
print("Coordinate ranges:")
print(f"  Latitude range: {restaurants['location/lat'].min():.6f} to {restaurants['location/lat'].max():.6f}")
print(f"  Longitude range: {restaurants['location/lng'].min():.6f} to {restaurants['location/lng'].max():.6f}")

# Check restaurant categories
print("\n" + "="*80)
print("Top 10 restaurant categories:")
print(restaurants['categoryName'].value_counts().head(10))

Examining restaurants data...

Records with missing coordinates: 0
Records with valid coordinates: 280

Coordinate ranges:
  Latitude range: 29.498436 to 29.725550
  Longitude range: 34.911016 to 35.006642

Top 10 restaurant categories:
categoryName
Restaurant              106
Fast food restaurant     23
Italian restaurant       15
Hamburger restaurant     14
Pizza restaurant         14
Asian restaurant         11
Pizza delivery            9
Shawarma restaurant       5
Sushi restaurant          4
Buffet restaurant         4
Name: count, dtype: int64


In [7]:
print("\nProcessing restaurants for spatial matching...\n")
print("="*80)

# Create point geometries from lat/lon
restaurants['geometry'] = restaurants.apply(
    lambda row: Point(row['location/lng'], row['location/lat'])
    if pd.notna(row['location/lat']) and pd.notna(row['location/lng'])
    else None,
    axis=1
)

# Create GeoDataFrame with WGS84 (lat/lon coordinate system)
gdf_restaurants = gpd.GeoDataFrame(
    restaurants,
    geometry='geometry',
    crs='EPSG:4326'  # lat/lon coordinate system
)

# Remove records without coordinates
gdf_restaurants = gdf_restaurants[gdf_restaurants['geometry'].notna()]

print(f"✓ Created {len(gdf_restaurants)} point geometries")

# Convert to the same CRS as statistical areas (EPSG:2039)
print(f"\nConverting coordinate systems...")
print(f"  Restaurants CRS: {gdf_restaurants.crs} (WGS84 lat/lon)")
print(f"  Statistical areas CRS: {gdf_areas.crs} (Israeli TM Grid)")

gdf_restaurants_transformed = gdf_restaurants.to_crs(gdf_areas.crs)
print(f"✓ Restaurants converted to {gdf_restaurants_transformed.crs}")

print("\n" + "="*80)
print("Performing spatial join (matching restaurants to statistical areas)...")

# Spatial join
restaurants_result = gpd.sjoin(
    gdf_restaurants_transformed,
    gdf_areas[['STAT_2022', 'geometry']],
    how='left',
    predicate='within'
)

# Count results
matched = restaurants_result['STAT_2022'].notna().sum()
unmatched = restaurants_result['STAT_2022'].isna().sum()

print(f"\n✓✓✓ Spatial join complete!")
print(f"\n  ✓ Matched: {matched} restaurants")
print(f"  ✗ Unmatched: {unmatched} restaurants")

if unmatched > 0:
    print(f"\n⚠ Warning: {unmatched} restaurants could not be matched")
    print("   (These might be outside Eilat's statistical area boundaries)")

print("\n" + "="*80)
print("Distribution of restaurants by statistical area:")
distribution = restaurants_result['STAT_2022'].value_counts().sort_index()
print(distribution)

print("\n" + "="*80)
print("Summary statistics:")
if matched > 0:
    print(f"  Total restaurants: {len(restaurants_result)}")
    print(f"  Statistical areas with restaurants: {restaurants_result['STAT_2022'].nunique()}")
    print(f"  Average restaurants per area: {matched / restaurants_result['STAT_2022'].nunique():.1f}")
    print(f"  Max restaurants in one area: {distribution.max()}")
    print(f"  Min restaurants in one area: {distribution.min()}")

# Check which statistical areas have NO restaurants
all_stat_areas = set(gdf_areas['STAT_2022'].values)
areas_with_restaurants = set(restaurants_result['STAT_2022'].dropna().values)
empty_areas = sorted(all_stat_areas - areas_with_restaurants)

if empty_areas:
    print(f"\n⚠ Statistical areas with NO restaurants: {empty_areas}")
else:
    print(f"\n✓ All statistical areas have at least one restaurant")



Processing restaurants for spatial matching...

✓ Created 280 point geometries

Converting coordinate systems...
  Restaurants CRS: EPSG:4326 (WGS84 lat/lon)
  Statistical areas CRS: EPSG:2039 (Israeli TM Grid)
✓ Restaurants converted to EPSG:2039

Performing spatial join (matching restaurants to statistical areas)...

✓✓✓ Spatial join complete!

  ✓ Matched: 278 restaurants
  ✗ Unmatched: 2 restaurants

⚠ Warning: 2 restaurants could not be matched
   (These might be outside Eilat's statistical area boundaries)

Distribution of restaurants by statistical area:
STAT_2022
11.0    106
12.0     20
13.0     49
14.0     14
15.0     10
16.0     37
17.0      2
18.0      3
19.0      2
21.0      2
22.0      1
24.0      2
31.0      5
33.0     17
42.0      2
43.0      2
44.0      1
45.0      2
46.0      1
Name: count, dtype: int64

Summary statistics:
  Total restaurants: 280
  Statistical areas with restaurants: 19
  Average restaurants per area: 14.6
  Max restaurants in one area: 106
  Min re

In [8]:
print("Creating map with ALL restaurants (matched + unmatched)...\n")

# Convert back to WGS84 for Folium
gdf_areas_wgs84 = gdf_areas.to_crs('EPSG:4326')

# Calculate center
center_lat = restaurants_result['location/lat'].mean()
center_lon = restaurants_result['location/lng'].mean()

print(f"Map center: {center_lat:.6f}, {center_lon:.6f}")

# Create base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Color scheme for areas
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
          'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
          'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray',
          'yellow', 'lightpink', 'olive', 'teal', 'navy', 'brown']

print("Adding statistical areas...")

# Add statistical areas
for idx, row in gdf_areas_wgs84.iterrows():
    stat_code = int(row['STAT_2022'])
    color = colors[idx % len(colors)]

    restaurant_count = len(restaurants_result[restaurants_result['STAT_2022'] == stat_code])

    popup_text = f"""
    <b>Statistical Area: {stat_code}</b><br>
    Restaurants: {restaurant_count}<br>
    Area: {row['Shape__Are']:.0f} m²
    """

    folium.GeoJson(
        row.geometry,
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.3
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

    # Add label
    centroid = row.geometry.centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(html=f"""
            <div style="
                font-size: 12px;
                font-weight: bold;
                color: black;
                background-color: white;
                border: 2px solid black;
                border-radius: 5px;
                padding: 2px 5px;
            ">{stat_code}</div>
        """)
    ).add_to(m)

# Separate matched and unmatched
matched_restaurants = restaurants_result[restaurants_result['STAT_2022'].notna()]
unmatched_restaurants = restaurants_result[restaurants_result['STAT_2022'].isna()]

print(f"Adding {len(matched_restaurants)} MATCHED restaurants (BLUE)...")

# Add MATCHED restaurants as BLUE markers
for idx, row in matched_restaurants.iterrows():
    popup_text = f"""
    <b>{row['title']}</b><br>
    Category: {row['categoryName']}<br>
    Score: {row['totalScore']}<br>
    Address: {row['street']}<br>
    <b>✓ Statistical Area: {int(row['STAT_2022'])}</b>
    """

    folium.CircleMarker(
        location=[row['location/lat'], row['location/lng']],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

print(f"Adding {len(unmatched_restaurants)} UNMATCHED restaurants (RED)...")

# Add UNMATCHED restaurants as RED markers
for idx, row in unmatched_restaurants.iterrows():
    popup_text = f"""
    <b>{row['title']}</b><br>
    Category: {row['categoryName']}<br>
    Score: {row['totalScore']}<br>
    Address: {row['street']}<br>
    <b>✗ Outside statistical areas</b>
    """

    folium.CircleMarker(
        location=[row['location/lat'], row['location/lng']],
        radius=8,
        popup=folium.Popup(popup_text, max_width=300),
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.9,
        weight=3
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed;
            bottom: 50px; right: 50px; width: 220px; height: 120px;
            background-color: white; border:2px solid grey; z-index:9999;
            font-size:14px; padding: 10px">
<p><strong>Restaurants Legend</strong></p>
<p><span style="color:blue;">⬤</span> Matched (in statistical area)</p>
<p><span style="color:red;">⬤</span> Unmatched (outside boundaries)</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save map
print("\nSaving map...")
m.save('eilat_restaurants_all_map.html')
print("✓ Map saved as: eilat_restaurants_all_map.html")

files.download('eilat_restaurants_all_map.html')

print("\n✓✓✓ Map created!")
print(f"\nMap shows:")
print(f"  - BLUE dots ({len(matched_restaurants)}): Restaurants matched to statistical areas")
print(f"  - RED dots ({len(unmatched_restaurants)}): Restaurants outside boundaries")
print(f"  - Total: {len(restaurants_result)} restaurants")

# Show unmatched restaurants details
if len(unmatched_restaurants) > 0:
    print("\n" + "="*80)
    print("Unmatched restaurants details:")
    print(unmatched_restaurants[['cid', 'title', 'street', 'location/lat', 'location/lng']])

# Display in notebook
m

Creating map with ALL restaurants (matched + unmatched)...

Map center: 29.555661, 34.954093
Adding statistical areas...
Adding 278 MATCHED restaurants (BLUE)...
Adding 2 UNMATCHED restaurants (RED)...

Saving map...
✓ Map saved as: eilat_restaurants_all_map.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ Map created!

Map shows:
  - BLUE dots (278): Restaurants matched to statistical areas
  - RED dots (2): Restaurants outside boundaries
  - Total: 280 restaurants

Unmatched restaurants details:
                      cid                                title  \
93   16610670765719936151                       קיסוס - Kissos   
219  15022544564624736051  ספינת אוריוניה הפלגות ארועים מסיבות   

                        street  location/lat  location/lng  
93   Derech Pa'amei HaShalom 1     29.547652     34.953467  
219               HaMayim St 1     29.548390     34.959984  


In [9]:
print("Manually assigning unmatched restaurants to statistical area 11...\n")
print("="*80)

# Show the unmatched restaurants first
unmatched = restaurants_result[restaurants_result['STAT_2022'].isna()]
print("Unmatched restaurants:")
print(unmatched[['cid', 'title', 'street', 'location/lat', 'location/lng']])

# Assign stat_2022 = 11 to unmatched restaurants
restaurants_result.loc[restaurants_result['STAT_2022'].isna(), 'STAT_2022'] = 11

# Verify the assignment
print("\n" + "="*80)
print("After manual assignment:")
matched = restaurants_result['STAT_2022'].notna().sum()
unmatched_count = restaurants_result['STAT_2022'].isna().sum()

print(f"  ✓ Matched: {matched} restaurants")
print(f"  ✗ Unmatched: {unmatched_count} restaurants")

print("\n" + "="*80)
print("Updated distribution:")
distribution = restaurants_result['STAT_2022'].value_counts().sort_index()
print(distribution)

print("\n✓✓✓ All restaurants now have a statistical area!")
print(f"Statistical area 11 now has {distribution[11.0]} restaurants (including 2 manually assigned)")

Manually assigning unmatched restaurants to statistical area 11...

Unmatched restaurants:
                      cid                                title  \
93   16610670765719936151                       קיסוס - Kissos   
219  15022544564624736051  ספינת אוריוניה הפלגות ארועים מסיבות   

                        street  location/lat  location/lng  
93   Derech Pa'amei HaShalom 1     29.547652     34.953467  
219               HaMayim St 1     29.548390     34.959984  

After manual assignment:
  ✓ Matched: 280 restaurants
  ✗ Unmatched: 0 restaurants

Updated distribution:
STAT_2022
11.0    108
12.0     20
13.0     49
14.0     14
15.0     10
16.0     37
17.0      2
18.0      3
19.0      2
21.0      2
22.0      1
24.0      2
31.0      5
33.0     17
42.0      2
43.0      2
44.0      1
45.0      2
46.0      1
Name: count, dtype: int64

✓✓✓ All restaurants now have a statistical area!
Statistical area 11 now has 108 restaurants (including 2 manually assigned)


In [10]:
print("\nCreating map with ALL restaurants matched (all BLUE)...\n")

# Convert back to WGS84 for Folium
gdf_areas_wgs84 = gdf_areas.to_crs('EPSG:4326')

# Calculate center
center_lat = restaurants_result['location/lat'].mean()
center_lon = restaurants_result['location/lng'].mean()

print(f"Map center: {center_lat:.6f}, {center_lon:.6f}")

# Create base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Color scheme for areas
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
          'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
          'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray',
          'yellow', 'lightpink', 'olive', 'teal', 'navy', 'brown']

print("Adding statistical areas...")

# Add statistical areas
for idx, row in gdf_areas_wgs84.iterrows():
    stat_code = int(row['STAT_2022'])
    color = colors[idx % len(colors)]

    restaurant_count = len(restaurants_result[restaurants_result['STAT_2022'] == stat_code])

    popup_text = f"""
    <b>Statistical Area: {stat_code}</b><br>
    Restaurants: {restaurant_count}<br>
    Area: {row['Shape__Are']:.0f} m²
    """

    folium.GeoJson(
        row.geometry,
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.4
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

    # Add label
    centroid = row.geometry.centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(html=f"""
            <div style="
                font-size: 12px;
                font-weight: bold;
                color: black;
                background-color: white;
                border: 2px solid black;
                border-radius: 5px;
                padding: 2px 5px;
            ">{stat_code}</div>
        """)
    ).add_to(m)

print(f"Adding {len(restaurants_result)} restaurants (all matched - BLUE)...")

# Add ALL restaurants as BLUE markers (all are now matched)
for idx, row in restaurants_result.iterrows():
    popup_text = f"""
    <b>{row['title']}</b><br>
    Category: {row['categoryName']}<br>
    Score: {row['totalScore']}<br>
    Address: {row['street']}<br>
    <b>Statistical Area: {int(row['STAT_2022'])}</b>
    """

    if pd.notna(row['url']):
        popup_text += f"<br><a href='{row['url']}' target='_blank'>View Details</a>"

    folium.CircleMarker(
        location=[row['location/lat'], row['location/lng']],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

# Save map
print("\nSaving map...")
m.save('eilat_restaurants_final_map.html')
print("✓ Map saved as: eilat_restaurants_final_map.html")

files.download('eilat_restaurants_final_map.html')

print("\n✓✓✓ Final map created!")
print(f"\nMap shows:")
print(f"  - All {len(restaurants_result)} restaurants are now matched (BLUE)")
print(f"  - 2 restaurants manually assigned to area 11")
print(f"  - Click on dots for restaurant details")
print(f"  - Click on areas for statistics")

# Display in notebook
m


Creating map with ALL restaurants matched (all BLUE)...

Map center: 29.555661, 34.954093
Adding statistical areas...
Adding 280 restaurants (all matched - BLUE)...

Saving map...
✓ Map saved as: eilat_restaurants_final_map.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ Final map created!

Map shows:
  - All 280 restaurants are now matched (BLUE)
  - 2 restaurants manually assigned to area 11
  - Click on dots for restaurant details
  - Click on areas for statistics


In [12]:
print("Preparing final restaurants dataset for export...\n")
print("="*80)

# Create final dataframe with stat_2022 column
final_restaurants = restaurants.copy()

# Add stat_2022 column from the spatial join result
# Match by cid to ensure correct alignment
final_restaurants['stat_2022'] = None

for idx, row in restaurants_result.iterrows():
    final_restaurants.loc[final_restaurants['cid'] == row['cid'], 'stat_2022'] = row['STAT_2022']

# Convert stat_2022 to integer
final_restaurants['stat_2022'] = final_restaurants['stat_2022'].astype(int)

# Reorder columns - put stat_2022 after coordinates
columns_order = [
    'cid',
    'title',
    'description',
    'categoryName',
    'totalScore',
    'temporarilyClosed',
    'permanentlyClosed',
    'url',
    'website',
    'street',
    'location/lat',
    'location/lng',
    'stat_2022',  # New foreign key column
    'Activity times'
]

final_restaurants = final_restaurants[columns_order]

print("Final dataset preview:")
print(final_restaurants.head(10))

print("\n" + "="*80)
print("Summary:")
print(f"✓ Total restaurants: {len(final_restaurants)}")
print(f"✓ Columns: {len(final_restaurants.columns)}")
print(f"✓ New 'stat_2022' column added (foreign key to statistical areas)")
print(f"✓ All restaurants have statistical areas (2 manually assigned to area 11)")

# Show distribution
print("\n" + "="*80)
print("Distribution of restaurants by statistical area:")
print(final_restaurants['stat_2022'].value_counts().sort_index())

# Show top categories
print("\n" + "="*80)
print("Top 10 restaurant categories:")
print(final_restaurants['categoryName'].value_counts().head(10))

# Export to Excel
print("\n" + "="*80)
print("Exporting files...")

excel_filename = 'Eilat_restaurants_with_stat_2022.xlsx'
final_restaurants.to_excel(excel_filename, index=False, engine='openpyxl')
print(f"✓ Excel file created: {excel_filename}")

# Export to CSV (with UTF-8 encoding)
csv_filename = 'Eilat_restaurants_with_stat_2022.csv'
final_restaurants.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"✓ CSV file created: {csv_filename}")

print("\n" + "="*80)
print("Downloading files...")

# Download both files
files.download(excel_filename)
files.download(csv_filename)

print("\n✓✓✓ SUCCESS! Files downloaded!")
print("\nYou now have:")
print(f"  1. Excel file: {excel_filename} ({len(final_restaurants)} restaurants)")
print(f"  2. CSV file: {csv_filename} ({len(final_restaurants)} restaurants)")
print(f"  3. Interactive map: eilat_restaurants_final_map.html")
print("\nAll files include the 'stat_2022' foreign key column!")
print("Note: 2 restaurants were manually assigned to statistical area 11")

Preparing final restaurants dataset for export...

Final dataset preview:
                    cid                                              title  \
0  13748798895203813526                                  Burger King Ramon   
1  11503681463165581249                             בורגרים שדה תעופה רמון   
2   7627013372514451195                                             Omer's   
3  12420276880766813630                                      Pastory Eilat   
4   3210688865794418088                                  Branja Restaurant   
5   1361957468337808507                                        The Brewery   
6   6440275474264107117  Tikka Masala - טיקה מסאלה אילת | מסעדה הודית ב...   
7  10738057538163195435                                              Pedro   
8  15039416062149028287                                         Wide Heart   
9  16499955574222481190                                   מסעדת Papi Grill   

                                         description          categ

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ SUCCESS! Files downloaded!

You now have:
  1. Excel file: Eilat_restaurants_with_stat_2022.xlsx (280 restaurants)
  2. CSV file: Eilat_restaurants_with_stat_2022.csv (280 restaurants)
  3. Interactive map: eilat_restaurants_final_map.html

All files include the 'stat_2022' foreign key column!
Note: 2 restaurants were manually assigned to statistical area 11
